# Vorarbeiten

In [23]:
import pandas as pd
import numpy as np
#data visualization
import matplotlib.pyplot as plt

## Umgebungsvariablen

In [2]:
failure_datapath = '''D:/jupyter-notebooks/LANL_Earthquake_Prediction/failure/'''
train_data_path = '''C:/studium/studium/CAS_PML/Projekt_Arbeit/earthquake/Daten/all/train.csv'''
feature_data_path = '''C:/studium/studium/CAS_PML/Projekt_Arbeit/earthquake/Daten/all/earthquakeFeatures.csv'''

## Trainingsdaten laden

In [3]:
train_data = pd.read_csv(train_data_path, dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

# Feature Extraction

Die Testsegmente bestehen jeweils aus 150.000 Messpunkte. Aus diesem Grund setzten wir die Fenstergrösse auch auf 150000 Messpunkte.

In [4]:
def generate_featureRow(acoustic_data, time_to_failure):
    
    strain_feature = []
    strain_feature.append(acoustic_data.mean())
    strain_feature.append(acoustic_data.std())
    strain_feature.append(acoustic_data.kurtosis())
    strain_feature.append(acoustic_data.skew())
    strain_feature.append(acoustic_data.min())
    strain_feature.append(acoustic_data.max())
    strain_feature.append(acoustic_data.quantile(0.01, interpolation='midpoint'))
    strain_feature.append(acoustic_data.quantile(0.05,interpolation='midpoint'))
    strain_feature.append(acoustic_data.quantile(0.95,interpolation='midpoint'))
    strain_feature.append(acoustic_data.quantile(0.99,interpolation='midpoint'))
    strain_feature.append(acoustic_data.abs().max())
    strain_feature.append(acoustic_data.abs().mean())
    strain_feature.append(acoustic_data.abs().std())
    
    strain_feature.append(time_to_failure.values[-1])
    
    return strain_feature

Das Fenster, mit der grösse 150000), wird durch das Trainingssignal geschoben mit einer geplanten Überlappung von 80%

In [5]:
def generate_Features(train,step, window_size=150000, ):
    init_idx = 0
    features = []

    while (init_idx + window_size < len(train)):
        window = train.iloc[init_idx:init_idx + window_size]
        features.append(generate_featureRow(window.acoustic_data,window.time_to_failure))
        init_idx += step

    return pd.DataFrame(features,dtype=np.float64)

In [35]:
step = 50000
features = generate_Features(train_data,step)

In [33]:
features.shape

(6290, 14)

In [ ]:
# saving the dataframe 
df.to_csv(feature_data_path, header=False, index=False) 

# Create Model Workbench

## Imports

In [10]:
# Scaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# Model selection
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Modell
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor

## GradientBoosting

In [11]:
pipe = Pipeline([('preprocessing', StandardScaler()), ('classifier', GradientBoostingRegressor())])

In [31]:
param_grid = [
    {'classifier': [GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, max_depth=1,random_state=0, loss='ls')],
     'classifier__learning_rate':[0.1,0.2,0.3,0.4,0.5], 'preprocessing': [None]}]

In [29]:
'''param_grid = [
    {'classifier': [SVC()], 'preprocessing': [StandardScaler(), None],
     'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],
     'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]},
    {'classifier': [RandomForestClassifier(n_estimators=100)],
     'preprocessing': [None], 'classifier__max_features': [1, 2, 3]}]'''

"param_grid = [\n    {'classifier': [SVC()], 'preprocessing': [StandardScaler(), None],\n     'classifier__gamma': [0.001, 0.01, 0.1, 1, 10, 100],\n     'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100]},\n    {'classifier': [RandomForestClassifier(n_estimators=100)],\n     'preprocessing': [None], 'classifier__max_features': [1, 2, 3]}]"

In [34]:
X_train, X_test, y_train, y_test = train_test_split(
    features.iloc[:,0:13], features.iloc[:,-1], random_state=0)

grid = GridSearchCV(pipe, param_grid, scoring='neg_mean_absolute_error', cv=5)
grid.fit(X_train, y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))
print("Test-set: mean absolute error : {:.2f}".format(mean_absolute_error(y_test, grid.predict(X_test))))


Best params:
{'classifier': GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=1, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=1000, n_iter_no_change=None, presort='auto',
             random_state=0, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False), 'classifier__learning_rate': 0.1, 'preprocessing': None}

Best cross-validation score: -2.16
Test-set score: -2.07
Test-set: mean absolute error : 2.07


In [ ]:
https://scikit-learn.org/stable/auto_examples/ensemble/plot_gradient_boosting_regression.html#sphx-glr-auto-examples-ensemble-plot-gradient-boosting-regression-py

# Open Points

gridsearch mit mae?

# Future Ideas

Reccurent Neuronal Net: LSTM